ACCESS TO S3

In [ ]:
import boto3

conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket_name)['Contents']
for f in contents:
    print(f['Key'])

: 

Create the Kubeflow pipeline

In [ ]:
# The unzipping function 
import zipfile
import os

def unzip_func(bucket_zipfile_path):
    """Unzips a file. 
    Parameters:
    bucket_zipfile_path : str, path to the zipfile in the bucket
    """
    # Set up connection
    s3_resource = boto3.client('s3')
    path_bucket = "bucket_name" # Change it to your bucket name
    path_to_move_file = "destination_subfolder/" # Change it to your 
                                                 # destination subfolder
    
    # Create local targets
    os.makedirs("./data", exist_ok=True)
    os.makedirs("./unzipped_data", exist_ok=True)
    
    # Download zipfile
    boto3.resource('s3').Object(path_bucket, bucket_zipfile_path).\
                        download_file(Filename="./data/zipfile.zip")
    
        # Unzip
    for zip in os.listdir("./data"):
        with zipfile.ZipFile(os.path.join("./data", zip), 'r') as file:
            file.extractall("./unzipped_data")
            
    # Upload
    for file in os.listdir("./unzipped_data"):
        output_path = path_to_move_file + file
        s3_resource.upload_file(os.path.join("./unzipped_data", file), 
                                path_bucket, output_path)
        
    # Return S3 path to last extracted csv file
    return output_path


: 

In [ ]:
# reader function 
import pandas as pd
from io import StringIO

def csv_s3_reader(bucket_name, 
                  csv_path, 
                  sep=";", 
                  decimal=",", 
                  encoding="utf-8"):
  """Reads a csv file from S3.
  Parameters:
  bucket_name : str, S3 bucket name,
  csv_path : str, path to your csv file in your bucket
  sep : str, parameter for pd.read_csv(),
  decimal : str, parameter for pd.read_csv(),
  encoding : str, parameter for pd.read_csv()
  
  Change sep, decimal and encoding depending on your csv's formatting
  """
  
  # Set up connection and download file
  csv_obj = boto3.client('s3').get_object(Bucket=bucket_name, Key=csv_path)
  body = csv_obj['Body']
  csv_string = body.read().decode(encoding)
  
  # Read csv
  df = pd.read_csv(StringIO(csv_string), sep=sep, decimal=decimal, error_bad_lines=False, encoding=encoding)
  
  return df

In [ ]:
# Reformat python functions to component functions to be  understandable by Kubeflow
import kfp
import kfp.components as comp

# First function
def unzip_files(bucket_zipfile_path : str, 
                output_string : comp.OutputPath(str)) :
    
    import boto3
    import os
    from io import BytesIO
    import zipfile
 
    # Set up connection
    s3_resource = boto3.client('s3')
    path_bucket = "bucket_name" # Change it to your bucket name
    path_to_move_file = "destination_subfolder/" # Change it to your 
                                                 # destination subfolder
    
    # Create local targets
    os.makedirs("./data", exist_ok=True)
    os.makedirs("./unzipped_data", exist_ok=True)
    
    # Download zipfile
    boto3.resource('s3').Object(path_bucket, bucket_zipfile_path).\
                        download_file(Filename="./data/zipfile.zip")
    # Unzip
    for zip in os.listdir("./data"):
        with zipfile.ZipFile(os.path.join("./data", zip), 'r') as file:
            file.extractall("./unzipped_data")
    
    # Upload
    for file in os.listdir("./unzipped_data"):
        output_path = path_to_move_file + file
        s3_resource.upload_file(os.path.join("./unzipped_data", file), 
                                path_bucket, output_path)
    
    # Create Artifact : S3 path to last extracted csv file    
    with open(output_string, 'w') as writer:
        writer.write(output_path)
        
# 2nd function
def csv_s3_reader(bucket_name : str, 
                  csv_path : comp.InputPath(str), 
                  sep : str, 
                  decimal : str, 
                  encoding : str, 
                  output_csv:comp.OutputPath('CSV')):
    

    from io import StringIO
    import pandas as pd
    import boto3
    import os
     # Set up connection and download file
    with open(csv_path, 'r') as reader:
        line = reader.readline()
        csv_obj = boto3.client('s3').get_object(Bucket=bucket_name, Key=line)
    body = csv_obj['Body']
    csv_string = body.read().decode(encoding)
    
    # Read csv
    df = pd.read_csv(StringIO(csv_string), sep=sep, decimal=decimal, 
                     error_bad_lines=False, encoding=encoding)
    
    df.to_csv(output_csv, index=True, header=True)